# Arcaico-Moderno: Test

## Melhor classificador encontrado

O melhor classificador encontrado pelas pipelines é -->    feature:CHAR-NGram + force-alpha==True

Melhor acucácia encontrada:  0.8631112370775755
Melhores parametros encontrados:  {'classifier__alpha': 0.1, 'classifier__force_alpha': True, 'kbest__k': 10000, 'vect__analyzer': 'char', 'vect__ngram_range': (4, 4)}

## Imports

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import ParameterGrid

### Import dataframe

In [2]:
import numpy as np
import pandas as pd

sep = ";"
dec = ","
quotech = "\""
encoding = "utf-8"
encoding_test = "latin-1"


EP_dir = "EP1"
test_dir = "test-no-labels"
CSV_input_name = "train_arcaico_moderno.csv"
CSV_test_input_name = "test_arcaico_moderno.csv"
path_to_archive = f"./Traindata/{EP_dir}/{CSV_input_name}"
path_to_test_archive = f"./Testdata/{test_dir}/{CSV_test_input_name}"

df = pd.read_csv(path_to_archive, na_values=['na'],
sep=sep,
decimal=dec,
quotechar=quotech,
encoding=encoding,
encoding_errors='strict')
print(df.shape)
print(df.columns)

(36884, 2)
Index(['text', 'style'], dtype='object')


## Melhor: Char-NGram + force-alpha=True

In [3]:
NGram_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('kbest', SelectKBest(score_func=chi2)),
    ('classifier', MultinomialNB()),
]) 

CHAR_NGram_parameters_BEST = { 
    'vect__ngram_range': [(4, 4)],
    'vect__analyzer': ["char"],
    'kbest__k': [10000],
    'classifier__force_alpha': [True],
    'classifier__alpha': [0.1],
}


CHAR_NGram_classifier_BEST = GridSearchCV(NGram_pipeline, CHAR_NGram_parameters_BEST, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
CHAR_NGram_classifier_BEST.fit(df["text"].fillna(""), df["style"].values)

print("Acurácia média:", CHAR_NGram_classifier_BEST.best_score_)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Acurácia média: 0.8631112370775755


## Controle de reprodutibilidade

A sessão de código abaixo deve ser configurada com os parâmetros da solução ótima obtida através de experimentação, e executada somente para validar a reprodutibilidade da entrega.

Saída limpa significa que não houve quebra de reprodutibilidade com os resultados desenvolvidos ao longo do trabalho.

In [4]:
import math

BEST_accuracy = CHAR_NGram_classifier_BEST.best_score_
control_accuracy = 0.8587735846560302

if not math.isclose(BEST_accuracy, control_accuracy, rel_tol=1e-11, abs_tol=1e-11):
    print(f"Quebra de controle da Acurácia: BEST Acc. é {BEST_accuracy} e Control Acc é {control_accuracy}")

Quebra de controle da Acurácia: BEST Acc. é 0.8631112370775755 e Control Acc é 0.8587735846560302


## Teste

Dados do teste.

In [9]:
df_test = pd.read_csv(path_to_test_archive, na_values=['na'],
sep=sep,
decimal=dec,
quotechar=quotech,
encoding=encoding_test,
encoding_errors='strict')
print(df_test.shape)
print(df_test.columns)

(9222, 1)
Index(['text'], dtype='object')


## Teste aplicado

In [15]:
X_test = df_test["text"].fillna("")

previsoes = CHAR_NGram_classifier_BEST.predict(X_test)

df_test['style_pred'] = previsoes

print(df_test['style_pred'].value_counts())

print("\n🔍 Amostra das previsões:")
print("-" * 60)
for i in range(min(3, len(df_test))):
    texto = df_test.iloc[i]['text']
    estilo = df_test.iloc[i]['style_pred']
    print(f"Texto {i+1}: {texto[:80]}...")
    print(f"Estilo previsto: {estilo}")
    print("-" * 40)

style_pred
moderno    4646
arcaico    4576
Name: count, dtype: int64

🔍 Amostra das previsões:
------------------------------------------------------------
Texto 1: E o Senhor disse a Caim: Por que te iraste? E por que descaiu o teu semblante?...
Estilo previsto: arcaico
----------------------------------------
Texto 2: E o anunciaram a Saul; dizendo: Eis que Davi está em Naiote; em Ramá....
Estilo previsto: arcaico
----------------------------------------
Texto 3: E o mordomo disse consigo: Que farei; pois que o meu senhor me tira a mordomia? ...
Estilo previsto: arcaico
----------------------------------------
